# Lab Assignment 2 Scientific Computing

Nick Boon & Marleen Rijksen

In [ ]:
import numpy as np
import time

from matplotlib import pyplot as plt
from matplotlib import animation,rc

from IPython.display import HTML
rc('animation',html='html5')

from numba import jit
from itertools import compress

%matplotlib inline

### K. Implementation of Diffusion Limited Aggregation (DLA) using boolean array

In [ ]:
@jit(nopython=True)
def diff_timestep_sor(cur, epsilon, omega, obj_array):
    """
    Performs a single timestep of the diffusion equation for
    Successive Over Relaxation method
    """
    Ny, Nx = cur.shape
    conv_bool = True
    
    # iterate over the domain
    for y in range(1,Ny-1):
        for x in range(0,Nx):
            prev = cur[y,x]
            left = x-1 if not x==0 else Nx-1
            right = x+1 if not x==Nx-1 else 0
            
            if obj_array[y,x]:
                cur[y,x] = 0
            else:
                cur[y,x] = omega/4 * (cur[y-1,x]+cur[y+1,x]+cur[y,right]+cur[y,left])\
                       + (1-omega) * cur[y,x]
                #do not allow negative values (would also create imaginary numbers if 
                #the power to eta is taken, as this does not need to be a integer)
                if cur[y,x] < 0:
                    cur[y,x] = 0
            
            # convergence condition
            if abs(prev-cur[y,x]) > epsilon:
                conv_bool = False
    return cur, conv_bool

@jit(nopython=True)
def DLA(cur,obj_array,neighbour_array,steps,epsilon,eta,omega):
    """
    Performs DLA algorithm
    
    Input:
    cur              NxM array containing concentration at every site
    obj_array        boolean NxM array. True if site is object
    neighbour_array  boolean NxM array. True if site is neighbour or object
    steps            number of iterations
    epsilon          convergence parameter in SOR
    eta              DLA exponent factor
    omega            relaxation factor in SOR
    
    Output:
    cur
    obj_array
    neighbour_array
    """
    for i in range(steps):
        #diffusion step
        conv_bool = False
        while not conv_bool:
            cur, conv_bool = diff_timestep_sor(cur, epsilon,\
                                               omega, obj_array)
        
        #obtain probability at candidate sites
        y,x = np.where(neighbour_array * ~obj_array)
        c_list = np.zeros(len(y))
        for i in range(len(y)):
            c_list[i] = cur[y[i],x[i]] ** eta
        
        #get sites to grow
        c_list = c_list / np.sum(c_list)
        rand = np.random.random(len(c_list))
        grow_list = c_list > rand
        #get coordinates of growing sites
        y = y[grow_list]
        x = x[grow_list]
        
        #add all growing sites to the arrays
        for i in range(len(x)):
            obj_array, neighbour_array = update_arrays(obj_array,\
                                                       neighbour_array,\
                                                       (y[i],x[i]))
            
    return cur,obj_array, neighbour_array

def generate_initial(shape,obj_loc,boundary_cond):
    """
    Generate initial concentration and object arrays
    
    Assume origin at upper-left of matrix
    
    Input:
    shape           shape of the space (y,x)
    obj_loc         tuple with location of first object (y,x)
    boundary_cond   tuple of boundary value at (y = 0, y = ymax)
    
    Output:
    cur              NxM array containing concentration at every site
    obj_array        boolean NxM array. True if site is object
    neighbour_array  boolean NxM array. True if site is neighbour or object
    """
    
    #init concentration array
    size_y, size_x = shape               #split tuple
    cur = np.zeros((size_x,size_y))      #init empty array
    down, up = boundary_cond             #split boundary_cond tuple
    cur[:] = np.linspace(down,up,size_y) #fill concentration array with linear decrease
    cur = cur.T                          #transpose to get y,x-matrix
    
    obj_array = np.full(shape, False)
    neighbour_array = np.full(shape,False)
    
    obj_array,neighbour_array = update_arrays(obj_array,neighbour_array,obj_loc)
    
    return cur,obj_array,neighbour_array

def show_object(cur,obj_array,neighbour_array,figsize=(10,8),\
                background=True,neighbour=True):
    """
    Generates figure, showing the concentration with the object and candidate sites. 
    Returns the figure and axes used.
    """
    
    #init figure, axes
    fig,ax = plt.subplots(figsize=figsize)
    
    #plot background or set limits correctly
    if background:
        cb = ax.imshow(cur,origin='lower')
        plt.colorbar(cb)

    #plot the object and candidate sites
    
    #plot neighbours first, as a filled circle will overlap an open circle for when
    #a neighbour site is also part of the object
    if neighbour:
        y,x = np.where(neighbour_array)
        ax.scatter(x,y,s=20, facecolors='none', edgecolors='k')
    
    #plot object
    y,x = np.where(obj_array)
    ax.scatter(x,y,s=20, facecolors='k', edgecolors='k')
    
    ymax, xmax = cur.shape
    ax.set_xlim([-0.5,xmax-0.5])
    ax.set_ylim([-0.5,ymax-0.5])
    
    return fig,ax

### L. Implementation of Monte Carlo Diffusion Limited Aggregation (MCDLA)

In [ ]:
@jit(nopython=True)
def random_walk(obj_array,neighbour_array,pos,stick_prob):
    ymax, xmax = np.array(neighbour_array.shape)-1
    y, x = pos
    
    while True:
        #choose random direction
        direction = np.random.randint(0,4)

        #move in direction, but revert if move into object
        if direction == 0:
            #left
            if x == 0:
                x = xmax
            else:
                x -= 1
            if obj_array[y,x]:
                #revert move
                if x == xmax:
                    x = 0
                else:
                    x += 1
                    
        elif direction == 1:
            #right
            if x == xmax:
                x = 0
            else:
                x += 1
            if obj_array[y,x]:
                #revert move
                if x == 0:
                    x = xmax
                else:
                    x -= 1
                    
        elif direction == 2:
            #up
            y += 1
            if y > ymax:
                break;
            if obj_array[y,x]:
                #revert move
                y -= 1
                
        elif direction == 3:
            #down
            y -= 1
            if y < 0:
                break;
            if obj_array[y,x]:
                #revert move
                y += 1
            
        if neighbour_array[y,x]:
            if np.random.random() < stick_prob:
                #break from while-loop as random_walker is now part of object
                break;
        
    pos = (y,x)
        
    return pos

@jit(nopython=True)
def update_arrays(obj_array,neighbour_array,add_object):
    ymax,xmax = np.array(obj_array.shape)-1
    y,x = add_object
    
    obj_array[y,x] = True
    
    if y < ymax:
        neighbour_array[y+1,x] = True
    if y > 0:
        neighbour_array[y-1,x] = True
    if x == 0:
        neighbour_array[y,-1] = True
        neighbour_array[y,0:2] = True
    elif x == xmax:
        neighbour_array[y,0] = True
        neighbour_array[y,x-1:x+1] = True
    else:
        neighbour_array[y,x-1:x+2] = True
    
    return obj_array, neighbour_array

@jit(nopython=True)
def DLA_MC(obj_array,neighbour_array,steps,stick_prob=1):
    ymax, xmax = np.array(obj_array.shape)-1
    for i in range(steps):
        #generate random walker at upper boundary
        pos = (ymax, np.random.randint(0,xmax))
        
        #get final position of random walker
        pos = random_walk(obj_array,neighbour_array,pos,stick_prob)
        
        #get y-coordinate
        y = pos[0]
        
        #if random walker is inside grid, add to obj_dict
        if y <= ymax and y >= 0:
            obj_array,neighbour_array = update_arrays(obj_array,neighbour_array,pos)
    
    return obj_array,neighbour_array

def generate_initial_MC(shape,obj_loc):
    
    obj_array = np.full(shape, False)
    neighbour_array = np.full(shape,False)
    
    obj_array, neighbour_array = update_arrays(obj_array,neighbour_array,obj_loc)
    
    return obj_array, neighbour_array

def show_object_MC(obj_array,neighbour_array,figsize=(10,8),neighbour=True):
    """
    Generates figure, showing the concentration with the object
    and candidate sites. Returns the figure and axes used.
    """
    
    #init figure, axes
    fig,ax = plt.subplots(figsize=figsize)
    
    ymax, xmax = obj_array.shape
    ax.set_xlim([-0.5,xmax-0.5])
    ax.set_ylim([-0.5,ymax-0.5])

    #plot the object and candidate sites

    #plot neighbours first, as a filled circle will overlap an open circle for when
    #a neighbour site is also part of the object
    if neighbour:
        y,x = np.where(neighbour_array)
        ax.scatter(x,y,s=20, facecolors='none', edgecolors='k')
    
    #plot object
    y,x = np.where(obj_array)
    ax.scatter(x,y,s=20, facecolors='k', edgecolors='k')
    
    return fig,ax

### Diffusion Limited Aggregation

In [ ]:
#Diffusion-limited Aggregation using boolean arrays
a = time.time()
cur,obj_array,neighbour_array = generate_initial((512,512),(0,256),(0,1))
cur,obj_array,neighbour_array = DLA(cur,obj_array,neighbour_array,12500,1E-4,1,1.96)
fig,ax = show_object(cur,obj_array,neighbour_array,(10,10),\
                     background=True,neighbour=False)
ax.set_title("Object, candidates and concentration")
ax.set_xlabel("x")
ax.set_ylabel("y")
print("TIME: %s"%(time.time()-a))

### Animated Diffusion Limited Aggregation

In [ ]:
# animation function.  This is called sequentially
def animate(i):
    global cur, obj_array, neighbour_array, cb, sc1
    
    cur, obj_array, neighbour_array = DLA(cur,obj_array,neighbour_array,\
                                          NUM_STEPS_PER_FRAME,1E-4,ETA,1.93)
    cb.set_data(cur)
    y,x = np.where(obj_array)
    sc1 = ax.scatter(x,y,s=15, facecolors='k', edgecolors='k')
    return cb, sc1
    
# set some parameters
NUM_STEPS_PER_FRAME = 5
NUM_FRAMES = 200
FRAME_INTERVAL = 25
ETA = 1

cur,obj_array,neighbour_array = generate_initial((100,100),(0,50),(0,1))
fig, ax = plt.subplots(figsize=(10,10))
cb = ax.imshow(cur,origin='lower')
plt.colorbar(cb)

ax.set_title("Diffusion Limited Aggregation\n$\eta$ = %.2f"%ETA)

ani = animation.FuncAnimation(fig, animate,
                               frames=NUM_FRAMES, interval=FRAME_INTERVAL, blit=True)

#close the figure, as we don't want to show that...
plt.close(fig)
#instead show the animation!
ani

### Monte Carlo Diffusion Limited Aggregation

In [ ]:
a = time.time()
obj_array,neighbour_array = generate_initial_MC((512,512),(0,256))
obj_array,neighbour_array = DLA_MC(obj_array,neighbour_array,int(7.5E6),1)
fig,ax = show_object_MC(obj_array,neighbour_array,(10,10),neighbour=False)
ax.set_title("Object, candidates and concentration")
ax.set_xlabel("x")
ax.set_ylabel("y")
print("TIME: %s"%(time.time()-a))

### Animated Monte Carlo Diffusion Limited Aggregation

In [ ]:
# animation function.  This is called sequentially
def animate(i):
    global obj_array, neighbour_array
    
    obj_array, neighbour_array = DLA_MC(obj_array,neighbour_array,\
                                          NUM_RANDOM_WALKERS_PER_FRAME,STICK_PROB)
    y,x = np.where(obj_array)
    sc1 = ax.scatter(x,y,s=15, facecolors='k', edgecolors='k')
    return cb, sc1
    
# set some parameters
NUM_RANDOM_WALKERS_PER_FRAME = int(2.5E2)
NUM_FRAMES = 250
FRAME_INTERVAL = 25
STICK_PROB = 1

SIZE = (100,100)
INITIAL = (0,50)

obj_array,neighbour_array = generate_initial_MC(SIZE,INITIAL)
fig, ax = plt.subplots(figsize=(10,10))
ax.set_ylim([-0.5,SIZE[1]-0.5])
ax.set_xlim([-0.5,SIZE[0]-0.5])

ax.set_title("Monte Carlo Diffusion Limited Aggregation\nSticking probability = %.2f"%STICK_PROB)

ani = animation.FuncAnimation(fig, animate,
                               frames=NUM_FRAMES, interval=FRAME_INTERVAL, blit=True)

#close the figure, as we don't want to show that...
plt.close(fig)
#instead show the animation!
ani